In [1]:
import sys
import os
from glob import glob
sys.path.append("../NEAT")
from NEATModels import NEATDynamic, nets
from NEATModels.config import dynamic_config
from NEATUtils import helpers
from NEATUtils.helpers import load_json
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

Using TensorFlow backend.


In [2]:
imagename = '/home/sancere/Kepler/FinalONEATTraining/NEATTest/Test.tif'
model_dir = '/home/sancere/Kepler/FinalONEATTraining/Microneatmodel/'
model_name = 'microseqnetbin2d56.h5'
division_categories_json = model_dir + 'MicroscopeCategories.json'
catconfig = load_json(division_categories_json)
division_cord_json = model_dir + 'MicroscopeCord.json'
cordconfig = load_json(division_cord_json)
model = NEATDynamic(None, model_dir , model_name,catconfig, cordconfig)
n_tiles = (4,4)

event_threshold = 0.8
iou_threshold = 0.3

In [ ]:
model.predict(imagename, n_tiles = n_tiles, event_threshold = event_threshold, iou_threshold = iou_threshold)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



  4%|▍         | 8/180 [00:54<18:57,  6.61s/it]

0.0 {'Normal': 2.2431366e-06, 'Division': 0.99999774, 'xstart': 142, 'ystart': 52, 'xcenter': 174.02440643310547, 'ycenter': 84.03023910522461, 'real_time_event': 8.0, 'box_time_event': 0.0, 'height': 19.35085678100586, 'width': 19.88811492919922, 'confidence': 1.0, 'realangle': 2, 'rawangle': 2}
0.0 {'Normal': 2.4588942e-06, 'Division': 0.9999975, 'xstart': 142, 'ystart': 52, 'xcenter': 174.01082229614258, 'ycenter': 83.98942756652832, 'real_time_event': 10.0, 'box_time_event': 0.0, 'height': 17.360580444335938, 'width': 17.26462173461914, 'confidence': 1.0, 'realangle': 2, 'rawangle': 2}
0.5520405514368348 {'Normal': 3.1554468e-05, 'Division': 0.9999684, 'xstart': 16, 'ystart': 56, 'xcenter': 48.1124267578125, 'ycenter': 88.05782699584961, 'real_time_event': 10.0, 'box_time_event': 0.0, 'height': 18.599443435668945, 'width': 19.324708938598633, 'confidence': 1.0, 'realangle': 2, 'rawangle': 2}
{'Normal': 3.1554468e-05, 'Division': 0.9999684, 'xstart': 16, 'ystart': 56, 'xcenter': 48.

  9%|▉         | 16/180 [01:47<17:52,  6.54s/it]

0.6691935178983965 {'Normal': 0.0021255717, 'Division': 0.9978744, 'xstart': 142, 'ystart': 56, 'xcenter': 173.99871635437012, 'ycenter': 88.00464630126953, 'real_time_event': 17.0, 'box_time_event': 0.0, 'height': 13.819417953491211, 'width': 16.747222900390625, 'confidence': 1.0, 'realangle': 2, 'rawangle': 2}
{'Normal': 0.0021255717, 'Division': 0.9978744, 'xstart': 142, 'ystart': 56, 'xcenter': 173.99871635437012, 'ycenter': 88.00464630126953, 'real_time_event': 17.0, 'box_time_event': 0.0, 'height': 13.819417953491211, 'width': 16.747222900390625, 'confidence': 1.0, 'realangle': 2, 'rawangle': 2}
0.9197623251110622 {'Normal': 0.004142119, 'Division': 0.9958579, 'xstart': 142, 'ystart': 52, 'xcenter': 174.03240585327148, 'ycenter': 83.97396659851074, 'real_time_event': 18.0, 'box_time_event': 0.0, 'height': 19.421323776245117, 'width': 17.42326545715332, 'confidence': 1.0, 'realangle': 2, 'rawangle': 2}
{'Normal': 0.004142119, 'Division': 0.9958579, 'xstart': 142, 'ystart': 52, 'xc

...

 20%|██        | 36/180 [03:57<15:42,  6.55s/it]

0.6965489865264234 {'Normal': 6.214177e-07, 'Division': 0.9999994, 'xstart': 79, 'ystart': 108, 'xcenter': 111.00841903686523, 'ycenter': 139.99957656860352, 'real_time_event': 37.0, 'box_time_event': 0.0, 'height': 18.421085357666016, 'width': 19.7385196685791, 'confidence': 1.0, 'realangle': 2, 'rawangle': 2}
{'Normal': 6.214177e-07, 'Division': 0.9999994, 'xstart': 79, 'ystart': 108, 'xcenter': 111.00841903686523, 'ycenter': 139.99957656860352, 'real_time_event': 37.0, 'box_time_event': 0.0, 'height': 18.421085357666016, 'width': 19.7385196685791, 'confidence': 1.0, 'realangle': 2, 'rawangle': 2}
0.37537952713939676 {'Normal': 1.0279921e-05, 'Division': 0.99998975, 'xstart': 79, 'ystart': 104, 'xcenter': 110.9959545135498, 'ycenter': 136.0007095336914, 'real_time_event': 37.0, 'box_time_event': 0.0, 'height': 16.379638671875, 'width': 18.013364791870117, 'confidence': 1.0, 'realangle': 2, 'rawangle': 2}
{'Normal': 1.0279921e-05, 'Division': 0.99998975, 'xstart': 79, 'ystart': 104, '

 22%|██▏       | 40/180 [04:24<15:29,  6.64s/it]

0.6873641153024971 {'Normal': 3.9039605e-06, 'Division': 0.99999607, 'xstart': 142, 'ystart': 56, 'xcenter': 174.03824615478516, 'ycenter': 88.03245162963867, 'real_time_event': 42.0, 'box_time_event': 0.0, 'height': 19.23514175415039, 'width': 23.01961898803711, 'confidence': 1.0, 'realangle': 2, 'rawangle': 2}
{'Normal': 3.9039605e-06, 'Division': 0.99999607, 'xstart': 142, 'ystart': 56, 'xcenter': 174.03824615478516, 'ycenter': 88.03245162963867, 'real_time_event': 42.0, 'box_time_event': 0.0, 'height': 19.23514175415039, 'width': 23.01961898803711, 'confidence': 1.0, 'realangle': 2, 'rawangle': 2}
0.0 {'Normal': 5.979063e-06, 'Division': 0.99999404, 'xstart': 142, 'ystart': 108, 'xcenter': 173.997953414917, 'ycenter': 140.07505416870117, 'real_time_event': 39.0, 'box_time_event': 0.0, 'height': 15.570300102233887, 'width': 18.716856002807617, 'confidence': 1.0, 'realangle': 2, 'rawangle': 2}
0.9174719076919622 {'Normal': 1.0914916e-05, 'Division': 0.99998903, 'xstart': 142, 'ystart

 38%|███▊      | 68/180 [07:36<12:21,  6.62s/it]